# Advanced techniques of creating and serving exceptions
---
[< __GO BACK__](https://github.com/VCauthon/Summary-OpenEdg-Pyhon-PCPP1/blob/main/1.Advanced-OOP/3.Advanced-Exceptions/Introduction.ipynb)

### Introduction

When Python executes a script and encounters a situation that it cannot cope with, it:

- Stops your program
- Creates a special kind of data, called an exception. Of course, this exception is an object

__Both__ of these activities are called __raising an exception__.


__¿What happens next?__ The exceptions expects something to take care of it and do something about it.
- If there is no such thing, the __program is forcibly terminated__ (and you will see an error message sent to the console).
- Otherwise, if the exception is __handled properly__, the program continues its execution.

Python categorizes all exceptions into a 63* build-in classes, each of which is responsible for a particular kind of problem. The classes are organized into a hierarchy, which means that some classes are subclasses of others.

__*NOTE__: The number of classes may vary depending on the version of Python you are using.

The hierarchy is presented in the picture below:

![Alt text](./media/exceptions_tree.png)

---

### Handling in code

When you suspect that the code may raise an exception, you should use the `try:` problematic_code `except` code block to surround the "problematic" piece of code. In effect, when the exception is raised, execution is not terminated, but the code following the `except` clause will try to handle the problem in an elegant way.

An example will be the following:

In [4]:
try:
    print(int("a"))  # An error will be raised here because we are trying to convert a string to a number (which we could actually do via ord but this is not the case).
except ValueError as e:
    print("An exception occurred: ", e)

An exception occurred:  invalid literal for int() with base 10: 'a'


---

### What contains the value "`e`" declared in `except ValueError as e`?

Basically, for this case e is an instance of ValueError with the context, that is, an object with the necessary attributes to indicate why the error originated.

The attributes of the object will vary depending on the exception raised, for example, the exception that controls when an error has arisen when importing a module (ImportError) has the following attributes:
- `name`: Indicates the module that has been tried to be imported.
- `path`: Indicates from where the module has been tried to be imported.

Additionally, as python exceptions are a sequence of inheritances it is important to note that there are several attributes that will always have an exception. Like `args` which is a tuple with the arguments that were passed to the exception constructor.

Another example, when an error related to the encoding is raised, the `UnicodeError` library comes into action. 

It has the following attributes:
- `encoding`:Tthe name of the encoding that raised the error.
- `reason`: String describing the specific codec error.
- `object`: The object the codec was attempting to encode or decode.
- `start`: The first index of invalid data in the object.
- `end`: The index after the last invalid data in the object.
---

### Introduction into chained exceptions

From python 3 onwards, chained exceptions are introduced. What are they for? Basically to know if, when handling an exception, another exception has been raised.

The context of both exceptions is stored as follows:
- `Implicit chaining`: When you raise a new exception without explicitly mentioning the previous exception.
- `Explicit chaining`: You can explicitly chain exceptions by using the from keyword when raising a new exception (basically when you add the `raise Exception('dummy')` __`from X`__).

This concept introduces two attributes in the created instances of an exception (`raise Exception as e`).

The attributes included are the following:
- `__context__`: Contains the implicit chaining.
- `__cause__`: Contains explicit chaining.

---

### Let's see an example of chained exceptions

As you can see in the code below, two exceptions are raised, the first one inside the try clause and a second error in the except clause.

This code will originate a chained exception and we can __identify this if we see the message that comes out between exceptions__.

`During handling of the above exception, another exception occurred:`

Ah there is the code i was talking about:

In [5]:
dummy_list = [1, 2] 

try:
    print(dummy_list[2])  # This will rise the first error (IndexError)
except IndexError as e:
    print(0/0)  # And this will rise the second error (ZeroDivisionError)

ZeroDivisionError: division by zero

---

### Wait where is the __context__ and __cause__ attributes?

From the above code we can see the context attribute if we catch the second exception and ask for __context__. This will return the detail of the first error raised.

Let's see this in an example:

In [13]:
dummy_list = [1, 2] 

try:
    print(dummy_list[2])  # This will rise the first error (IndexError)
except IndexError as e:
    
    try:
        print(0/0)  # And this will rise the second error (ZeroDivisionError)
    except ZeroDivisionError as e2:
        print('Inner exception (f):', e)
        print('Outer exception (e):', e2)
        print('Outer exception referenced:', e2.__context__)
        print('Is it the same object:', e2.__context__ is e)

Inner exception (f): list index out of range
Outer exception (e): division by zero
Outer exception referenced: list index out of range
Is it the same object: True


---

### And what about the __cause__ attribute?

This originates when we raise an error from the exception section including the details of what happened.

Let's rescue the previous code example:

In [16]:
class DummyException(Exception):
    pass

dummy_list = [1, 2] 

try:
    print(dummy_list[2])  # This will rise the first error (IndexError)
except IndexError as e:
    raise DummyException('This is a dummy exception') from e

DummyException: This is a dummy exception

The error message to highlight on the displayed is:

`The above exception was the direct cause of the following exception:`

This is like the previous one we have seen when performing an Implicit chaining (where we saw the detail of the previous exception through the context), but for Explicit chaining.

---

### Ok, but where is the use of the __cause__ attribute?

To do this, it would be necessary to check the dummy exception that has been raised and ask it for that attribute, which would return ... the arguments of the original IndexError

Code example:

In [18]:
class DummyException(Exception):
    pass

dummy_list = [1, 2] 

try:
    print(dummy_list[2])  # This will rise the first error (IndexError)
except IndexError as e:
    try:
        raise DummyException('This is a dummy exception') from e
    except DummyException as f:
        print(f.__cause__.args)  # Wow i other exception i can access the original exception

('list index out of range',)


---

### Where does the exception come from?

All Python exceptions have a traceback. This allows us to know in which line of code the error was raised, but...

Look what happens when we display the traceback of the last example:

In [19]:
class DummyException(Exception):
    pass

dummy_list = [1, 2] 

try:
    print(dummy_list[2])  # This will rise the first error (IndexError)
except IndexError as e:
    try:
        raise DummyException('This is a dummy exception') from e
    except DummyException as f:
        print(f.__traceback__)

What we are seeing is a traceback object, which now we may be wondering ... How can I use this?

Here luckily python has two build-in functions to be able to display the traceback on the screen.
- print_tb: Which displays the content on the screen.
- format_tb: Which allows us to format the traceback together with another string.

Let's see an example:

In [26]:
from traceback import print_tb, format_tb # Let's don't forget to import it!!

class DummyException(Exception):
    pass

dummy_list = [1, 2] 

try:
    print(dummy_list[2])  # This will rise the first error (IndexError)
except IndexError as e:
    try:
        raise DummyException('This is a dummy exception') from e
    except DummyException as f:
        print(f.__traceback__)
        print_tb(f.__traceback__)
        print(f"Hi {format_tb(f.__traceback__)}")

Hi ['  File "/tmp/ipykernel_7797/536246293.py", line 12, in <module>\n    raise DummyException(\'This is a dummy exception\') from e\n']


  File "/tmp/ipykernel_7797/536246293.py", line 12, in <module>
    raise DummyException('This is a dummy exception') from e


---
[< __GO BACK__](https://github.com/VCauthon/Summary-OpenEdg-Pyhon-PCPP1/blob/main/1.Advanced-OOP/3.Advanced-Exceptions/Introduction.ipynb)